In [1]:
import sys
import os
from pathlib import Path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('hey-database-tutorial')


api_key = os.getenv('OPENAI_API_KEY')
db_pwd = os.getenv('POSTGRES_PWD')

In [2]:
from src.config.config_loader import ConfigLoader

config = ConfigLoader.load_config(
    db_config_path=os.path.join(
        project_root, "configs", "DB_connections", "northwind_postgres.yaml"
    ),
    cache_config_path=os.path.join(
        project_root, "configs", "cache", "northwind_cache.yaml"
    ),
    sql_llm_config_path=os.path.join(
        project_root, "configs", "sql_llm", "openai_4o_mini.yaml"
    ),
    vector_store_config_path=os.path.join(
        project_root, "configs", "vector_store", "qdrant_northwind.yaml"
    ),
    prompt_config_path=os.path.join(project_root, "configs", "prompt.yaml"),
    metadata_config_path=os.path.join(project_root, "configs", "metadata_.yaml"),
    base_config_path=os.path.join(project_root, "configs", "base_config.yaml"),
)

In [3]:
from src.build.build_app_components import AppComponentsBuilder
app_components = AppComponentsBuilder(config).build()

/Users/giacomo/miniconda3/envs/heydatabase/lib/python3.10/site-packages/snowflake/sqlalchemy/base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)
/Users/giacomo/miniconda3/envs/heydatabase/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
INFO:hey-database:Connected to PostgreSQL database
INFO:hey-database:Initializing vector store components...
INFO:hey-database:Vector store components initialized successfully


In [4]:
from src.metadata.metadata_startup import (
    MetadataStartup,
    MetadataProcessor,
)

metadata_processor = MetadataProcessor(
    table_extractor=app_components.table_metadata_extractor,
    column_extractor=app_components.column_metadata_extractor,
    table_enhancer=app_components.table_metadata_enhancer,
    column_enhancer=app_components.column_metadata_enhancer,
)
metadata_manager = MetadataStartup(metadata_processor, app_components.cache)

In [5]:
metadata = metadata_manager.initialize_metadata()

INFO:hey-database:Found valid cached metadata.


In [6]:
metadata

Metadata(tables={'territories': TableMetadata(base_metadata=BaseTableMetadata(name='territories', columns=['territory_id', 'territory_description', 'region_id'], primary_keys=['territory_id'], foreign_keys=[{'constrained_columns': ['region_id'], 'referred_table': 'region', 'referred_columns': ['region_id']}], row_count=53), description='placeholder', keywords=['placeholder'], importance_score=0.8), 'order_details': TableMetadata(base_metadata=BaseTableMetadata(name='order_details', columns=['order_id', 'product_id', 'unit_price', 'quantity', 'discount'], primary_keys=['order_id', 'product_id'], foreign_keys=[{'constrained_columns': ['order_id'], 'referred_table': 'orders', 'referred_columns': ['order_id']}, {'constrained_columns': ['product_id'], 'referred_table': 'products', 'referred_columns': ['product_id']}], row_count=2155), description='placeholder', keywords=['placeholder'], importance_score=1.0), 'employee_territories': TableMetadata(base_metadata=BaseTableMetadata(name='employ

In [7]:
from src.store.vectorstore_startup import VectorStoreStartup

vector_store_startup = VectorStoreStartup(app_components.vector_store, app_components.vector_store_writer)
vector_store_startup.initialize(metadata)

INFO:hey-database:Initializing vector store collection
INFO:hey-database:Collection northwind exists with valid configuration
INFO:hey-database:Syncing metadata to vector store
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/em

True